<a href="https://colab.research.google.com/github/atik666/AugmmentationCode/blob/TransferLearning/robustTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from tensorflow import keras
from keras.preprocessing.image import load_img, ImageDataGenerator, img_to_array
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from matplotlib import pyplot as plt
import numpy as np

In [4]:
X_train1 = np.load("/content/drive/MyDrive/Colab/Main/X_train1.npy")
y_train1 = np.load("/content/drive/MyDrive/Colab/Main/y_train1.npy")
X_train2 = np.load("/content/drive/MyDrive/Colab/Aug/Gan_X_train.npy")
y_train2 = np.load("/content/drive/MyDrive/Colab/Aug/Gan_y_train.npy")


In [5]:
X_train = np.concatenate((X_train1, X_train2))
y_train = np.concatenate((y_train1, y_train2))

print('Number of images in X_train', X_train.shape[0])
print('Number of labels in y_train', y_train.shape[0])

Number of images in X_train 2000
Number of labels in y_train 2000


In [6]:
IMAGE_SHAPE = (224, 224, 3)
      
model_name = "inception_v3"
feature_extractor_model="https://tfhub.dev/google/tf2-preview/%s/feature_vector/4" %model_name

pretrained_model_without_top_layer = hub.KerasLayer(
    feature_extractor_model, input_shape=IMAGE_SHAPE, trainable=False)
       
num_class = 10

In [ ]:
model = tf.keras.Sequential([
  pretrained_model_without_top_layer,
  tf.keras.layers.Dense(num_class)
])

model.summary()          
         
model.compile(
  optimizer="adam",
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])
     
# Train your model using the early stopping callback
model.fit(X_train, y_train, batch_size = 32, epochs = 50)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 2048)              21802784  
_________________________________________________________________
dense (Dense)                (None, 10)                20490     
Total params: 21,823,274
Trainable params: 20,490
Non-trainable params: 21,802,784
_________________________________________________________________
Epoch 1/50
63/63 [==============================] - 37s 92ms/step - loss: 1.2868 - accuracy: 0.5885
Epoch 2/50
63/63 [==============================] - 5s 77ms/step - loss: 0.6061 - accuracy: 0.8100
Epoch 3/50
63/63 [==============================] - 5s 78ms/step - loss: 0.4497 - accuracy: 0.8705
Epoch 4/50
63/63 [==============================] - 5s 79ms/step - loss: 0.3693 - accuracy: 0.8945
Epoch 5/50
63/63 [==============================] - 5s 78ms/step - loss: 0.3159 - accuracy: 0.9160
Epoch

In [ ]:
# load = 1
# X_test = np.load("/content/drive/MyDrive/Colab/%sTest/%sX_test.npy" %(load, load))
# y_test = np.load("/content/drive/MyDrive/Colab/%sTest/%sy_test.npy" %(load, load))

#SNR Test
snr = 10
X_test = np.load("/content/drive/MyDrive/Colab/SNR_Test/%sX_test.npy" %snr)
y_test = np.load("/content/drive/MyDrive/Colab/SNR_Test/%sy_test.npy" %snr)

loss, acc= model.evaluate(X_test, y_test)
print('Test Accuracy: %f' % (acc*100))

from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)
conf = confusion_matrix(y_test, y_pred)
print(conf)
print('acc : ', np.trace(conf)/np.sum(conf)*100)